In [1]:
%matplotlib inline
import pandas as pd

path="C:\\Users\\nived\\Desktop\\Nivi ideapad Lenovo\\Nivoo SJ\\SJSU sem 2\\CS274\\project ideas\\nivedhitha book recomm\\data\\goodbooks-10k-master\\"
ratings = pd.read_csv( path+'small_ratings.csv' )
actual_ratings = pd.read_csv( path+'small_ratings.csv' )
#to_read = pd.read_csv( path+'to_read.csv' )
books = pd.read_csv( path+'small_books.csv' )


In [2]:
ratings.head()

,user_id,book_id,rating
0,2,26,4
1,2,33,4
2,4,70,4
3,4,18,5
4,4,27,5


In [3]:
books.head()

,book_id,books_count,authors,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5
0,1,272,Suzanne Collins,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317
1,2,491,"J.K. Rowling, Mary GrandPré",Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543
2,3,226,Stephenie Meyer,Twilight,"Twilight (Twilight, #1)",en-US,3.57,3866839,3916824,95009,456191,436802,793319,875073,1355439
3,4,487,Harper Lee,To Kill a Mockingbird,To Kill a Mockingbird,eng,4.25,3198671,3340896,72586,60427,117415,446835,1001952,1714267
4,5,1356,F. Scott Fitzgerald,The Great Gatsby,The Great Gatsby,eng,3.89,2683664,2773745,51992,86236,197621,606158,936012,947718


In [5]:
ratings_mtx_trust = ratings.pivot(index = 'user_id', columns ='book_id', values = 'rating').fillna(0)
ratings_mtx_trust = ratings.pivot(index = 'user_id', columns ='book_id', values = 'rating').fillna(0)
ratings_mtx_trust.head()
actualRatings = actual_ratings.pivot(index = 'user_id', columns ='book_id', values = 'rating').fillna(0)
actualRatings_mtx=actualRatings.as_matrix()
print(ratings_mtx_trust.shape)

(42, 87)


In [35]:
from math import *
  
def jaccard_similarity(x,y):
  
 intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
 #print intersection_cardinality
 union_cardinality = len(set.union(*[set(x), set(y)]))
 #print union_cardinality 
 return intersection_cardinality/float(union_cardinality)

def intersect(a, b):
    return list(set(a) & set(b))
    #print(intersect(b1, b2))

def jacra_similarity(x,y):
    sum_rate=0
    for i in x.index:
        #for each user's rating for book x and y
        if(x[i]!=0 and y[i]!=0):
            #then both books have been rated something by user i
            min_rate=min(x[i],y[i])
            max_rate=max(x[i],y[i])
            sum_rate+=min_rate/max_rate
    intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
    jacra=jaccard_similarity(x,y)*(sum_rate/intersection_cardinality)
    return jacra

def initialize_sim_mtx(method):
    if(method=='jacra'):
        unique_ISBN=ratings_mtx_trust.columns.unique()
        #print(len(unique_ISBN))
        sim_jacra_mtx=pd.DataFrame(index=unique_ISBN,columns=unique_ISBN).fillna(0.0).as_matrix()
        print(sim_jacra_mtx)
        return(sim_jacra_mtx)
    
def construct_sim_mtx(matrix,method):    
    for i,row in enumerate(matrix):
        for j,cell in enumerate(row):
            if i!=j :
                if method=='jacra':
                    ratings_isbn1=ratings_mtx_trust.iloc[:,i]
                    ratings_isbn2=ratings_mtx_trust.iloc[:,j]
                    sim=round(jacra_similarity(ratings_isbn1,ratings_isbn2),3)
                    if j+1 in normalize_books_trust:
                        sim_trust_mtx[i][j]=sim+(sim*normalize_books_trust[j+1])
                    else:
                        sim_trust_mtx[i][j]=sim
    print("done")

In [7]:
joined=ratings.join(books.set_index('book_id'), on='book_id')
joined.shape

(674, 17)

In [8]:
def get_pred(actualRatings_mtx,method):
    pred_rating=0
    if(method=='jacra'):
        print("predicting for jacra")
        predicted_ratings_matrix=ratings_mtx_trust.as_matrix()
        #actual_ratings_matrix=sratings_mtx_jacra.as_matrix()
        print("actual for jacra ",actualRatings_mtx)
        print("predicted for jacra ",predicted_ratings_matrix)
        for i,user in enumerate(actualRatings_mtx):
            # i is userid row is all books
            sum_user_j=0
            denom=0
            for j,book in enumerate(user):
                #should update in predicted_ratings_matrix[i][j]
                if(predicted_ratings_matrix[i][j]!=0):
                    sum_user_j+=sim_trust_mtx[i][j]*actualRatings_mtx[i][j]
                    denom+=sim_trust_mtx[i][j]
                    pred_rating=sum_user_j/denom
                #print("for i ,j", i,j," sum: ",sum_user_j," denom: ",denom," total rating: ",pred_rating)
                predicted_ratings_matrix[i][j]=pred_rating
    return predicted_ratings_matrix


In [9]:
book_author=joined[['user_id','book_id','rating','books_count','authors']]

In [10]:
book_author=book_author[book_author.user_id<=50]
book_author=book_author[book_author.book_id<=100]
book_author.shape

(674, 5)

In [11]:
author_trust={}
authors=[]
for auth in book_author['authors']:
    for a in auth.split(','):
        if a not in authors:
            authors.append(a)

In [13]:
for a in authors:
    author_trust[a]=0.0
print(author_trust)

{'Dan Brown': 0.0, 'Arthur Golden': 0.0, 'Orson Scott Card': 0.0, 'J.K. Rowling': 0.0, ' Mary GrandPré': 0.0, ' Rufus Beck': 0.0, 'Paulo Coelho': 0.0, ' Alan R. Clarke': 0.0, 'Michael Crichton': 0.0, 'Mark Twain': 0.0, ' John Seelye': 0.0, ' Guy Cardwell': 0.0, 'F. Scott Fitzgerald': 0.0, 'J.D. Salinger': 0.0, 'Kurt Vonnegut Jr.': 0.0, 'Yann Martel': 0.0, 'George Orwell': 0.0, 'Aldous Huxley': 0.0, 'Lois Lowry': 0.0, ' Erich Fromm': 0.0, ' Celâl Üster': 0.0, 'Markus Zusak': 0.0, 'Shel Silverstein': 0.0, 'Elie Wiesel': 0.0, ' Marion Wiesel': 0.0, 'John Grisham': 0.0, 'John Steinbeck': 0.0, 'William Golding': 0.0, 'Louisa May Alcott': 0.0, 'Khaled Hosseini': 0.0, 'Jeannette Walls': 0.0, 'Mark Haddon': 0.0, 'Jane Austen': 0.0, ' Tony Tanner': 0.0, ' Ros Ballaster': 0.0, 'Gabriel García Márquez': 0.0, ' Gregory Rabassa': 0.0, 'Harper Lee': 0.0, 'Barbara Kingsolver': 0.0, 'Steven D. Levitt': 0.0, ' Stephen J. Dubner': 0.0, 'Mary Wollstonecraft Shelley': 0.0, ' Percy Bysshe Shelley': 0.0, ' 

In [14]:
#6:40-7
maxnf=0.0 
for author,trust in author_trust.items():
    #print(author,trust)
    nf=0.0   
    for row in book_author.iterrows():
        #print(row[1].iloc[4])
        #print(row[1])
        if author in row[1].iloc[4] and row[1].iloc[2]>0:
            nf+=1
            if nf>maxnf:
                maxnf=nf
    t=nf/maxnf
    author_trust[author]=t
print(author_trust)
                

{'Dan Brown': 1.0, 'Arthur Golden': 0.5185185185185185, 'Orson Scott Card': 0.2962962962962963, 'J.K. Rowling': 1.0, ' Mary GrandPré': 1.0, ' Rufus Beck': 0.16091954022988506, 'Paulo Coelho': 0.12643678160919541, ' Alan R. Clarke': 0.12643678160919541, 'Michael Crichton': 0.04597701149425287, 'Mark Twain': 0.04597701149425287, ' John Seelye': 0.04597701149425287, ' Guy Cardwell': 0.04597701149425287, 'F. Scott Fitzgerald': 0.14942528735632185, 'J.D. Salinger': 0.20689655172413793, 'Kurt Vonnegut Jr.': 0.10344827586206896, 'Yann Martel': 0.1724137931034483, 'George Orwell': 0.40229885057471265, 'Aldous Huxley': 0.12643678160919541, 'Lois Lowry': 0.05747126436781609, ' Erich Fromm': 0.20689655172413793, ' Celâl Üster': 0.20689655172413793, 'Markus Zusak': 0.08045977011494253, 'Shel Silverstein': 0.16091954022988506, 'Elie Wiesel': 0.04597701149425287, ' Marion Wiesel': 0.04597701149425287, 'John Grisham': 0.022988505747126436, 'John Steinbeck': 0.12643678160919541, 'William Golding': 0.1

In [15]:
print(author_trust)

{'Dan Brown': 1.0, 'Arthur Golden': 0.5185185185185185, 'Orson Scott Card': 0.2962962962962963, 'J.K. Rowling': 1.0, ' Mary GrandPré': 1.0, ' Rufus Beck': 0.16091954022988506, 'Paulo Coelho': 0.12643678160919541, ' Alan R. Clarke': 0.12643678160919541, 'Michael Crichton': 0.04597701149425287, 'Mark Twain': 0.04597701149425287, ' John Seelye': 0.04597701149425287, ' Guy Cardwell': 0.04597701149425287, 'F. Scott Fitzgerald': 0.14942528735632185, 'J.D. Salinger': 0.20689655172413793, 'Kurt Vonnegut Jr.': 0.10344827586206896, 'Yann Martel': 0.1724137931034483, 'George Orwell': 0.40229885057471265, 'Aldous Huxley': 0.12643678160919541, 'Lois Lowry': 0.05747126436781609, ' Erich Fromm': 0.20689655172413793, ' Celâl Üster': 0.20689655172413793, 'Markus Zusak': 0.08045977011494253, 'Shel Silverstein': 0.16091954022988506, 'Elie Wiesel': 0.04597701149425287, ' Marion Wiesel': 0.04597701149425287, 'John Grisham': 0.022988505747126436, 'John Steinbeck': 0.12643678160919541, 'William Golding': 0.1

In [16]:
books_trust={}
for rating_row in ratings.iterrows():
    bid=rating_row[1].iloc[1]
    rate=rating_row[1].iloc[2]
    rated=0
    qt=0.0
    print("bid: ",bid," rating: ",rate)
    books_count=books['books_count'].iloc[bid-1]
    print("books_count for: ",books_count)
    if rate>0:
        rated+=1
    if rated>books_count:
        rated=books_count
    qt=rated/books_count
    books_trust[bid]=qt     
        

bid:  26  rating:  4
books_count for:  350
bid:  33  rating:  4
books_count for:  220
bid:  70  rating:  4
books_count for:  224
bid:  18  rating:  5
books_count for:  376
bid:  27  rating:  5
books_count for:  275
bid:  21  rating:  5
books_count for:  307
bid:  2  rating:  5
books_count for:  491
bid:  23  rating:  5
books_count for:  398
bid:  24  rating:  5
books_count for:  332
bid:  35  rating:  5
books_count for:  458
bid:  26  rating:  3
books_count for:  350
bid:  84  rating:  4
books_count for:  176
bid:  58  rating:  4
books_count for:  2277
bid:  5  rating:  4
books_count for:  1356
bid:  8  rating:  4
books_count for:  360
bid:  65  rating:  4
books_count for:  241
bid:  45  rating:  4
books_count for:  264
bid:  14  rating:  5
books_count for:  896
bid:  55  rating:  5
books_count for:  515
bid:  36  rating:  5
books_count for:  192
bid:  13  rating:  4
books_count for:  995
bid:  55  rating:  4
books_count for:  515
bid:  47  rating:  3
books_count for:  251
bid:  50  ra

bid:  41  rating:  4
books_count for:  159
bid:  40  rating:  3
books_count for:  185
bid:  33  rating:  4
books_count for:  220
bid:  50  rating:  5
books_count for:  45
bid:  32  rating:  4
books_count for:  373
bid:  14  rating:  5
books_count for:  896
bid:  22  rating:  4
books_count for:  183
bid:  38  rating:  4
books_count for:  167
bid:  100  rating:  3
books_count for:  39
bid:  94  rating:  5
books_count for:  555
bid:  85  rating:  3
books_count for:  81
bid:  95  rating:  4
books_count for:  2303
bid:  50  rating:  5
books_count for:  45
bid:  9  rating:  3
books_count for:  311
bid:  28  rating:  4
books_count for:  458
bid:  46  rating:  4
books_count for:  128
bid:  77  rating:  4
books_count for:  138
bid:  45  rating:  3
books_count for:  264
bid:  28  rating:  5
books_count for:  458
bid:  13  rating:  3
books_count for:  995
bid:  75  rating:  4
books_count for:  193
bid:  5  rating:  3
books_count for:  1356
bid:  37  rating:  4
books_count for:  474
bid:  13  rati

books_count for:  274
bid:  1  rating:  4
books_count for:  272
bid:  17  rating:  3
books_count for:  201
bid:  20  rating:  4
books_count for:  239
bid:  16  rating:  3
books_count for:  274
bid:  47  rating:  4
books_count for:  251
bid:  84  rating:  5
books_count for:  176
bid:  43  rating:  5
books_count for:  2568
bid:  11  rating:  5
books_count for:  283
bid:  17  rating:  4
books_count for:  201
bid:  1  rating:  3
books_count for:  272
bid:  1  rating:  4
books_count for:  272
bid:  39  rating:  4
books_count for:  101
bid:  31  rating:  5
books_count for:  183
bid:  39  rating:  5
books_count for:  101
bid:  1  rating:  3
books_count for:  272
bid:  39  rating:  4
books_count for:  101
bid:  31  rating:  5
books_count for:  183
bid:  39  rating:  4
books_count for:  101
bid:  38  rating:  2
books_count for:  167
bid:  67  rating:  3
books_count for:  197
bid:  46  rating:  4
books_count for:  128
bid:  22  rating:  3
books_count for:  183
bid:  31  rating:  4
books_count fo

In [17]:
print(books_trust[5])

0.0007374631268436578


In [18]:
new_books_trust={}
for row in book_author.iterrows():
    bid=row[1].iloc[1]
    authors=row[1].iloc[4].split(",")
    all_trusts=[]
    for a in authors:
        all_trusts.append(author_trust[a])
    print("all trusts: ",all_trusts)
    max_trust=max(all_trusts)
    print("max trusts: ",max_trust)
    bid_trust=books_trust[bid]
    print("bid trusts: ",bid, " ",bid_trust)
    tot_trust=bid_trust+max_trust
    print("total trusts: ",tot_trust)
    new_books_trust[bid]=tot_trust

all trusts:  [1.0]
max trusts:  1.0
bid trusts:  26   0.002857142857142857
total trusts:  1.002857142857143
all trusts:  [0.5185185185185185]
max trusts:  0.5185185185185185
bid trusts:  33   0.004545454545454545
total trusts:  0.523063973063973
all trusts:  [0.2962962962962963]
max trusts:  0.2962962962962963
bid trusts:  70   0.004464285714285714
total trusts:  0.300760582010582
all trusts:  [1.0, 1.0, 0.16091954022988506]
max trusts:  1.0
bid trusts:  18   0.0026595744680851063
total trusts:  1.002659574468085
all trusts:  [1.0, 1.0]
max trusts:  1.0
bid trusts:  27   0.0036363636363636364
total trusts:  1.0036363636363637
all trusts:  [1.0, 1.0]
max trusts:  1.0
bid trusts:  21   0.003257328990228013
total trusts:  1.003257328990228
all trusts:  [1.0, 1.0]
max trusts:  1.0
bid trusts:  2   0.002036659877800407
total trusts:  1.0020366598778003
all trusts:  [1.0, 1.0]
max trusts:  1.0
bid trusts:  23   0.002512562814070352
total trusts:  1.0025125628140703
all trusts:  [1.0, 1.0]
ma

bid trusts:  4   0.002053388090349076
total trusts:  0.1974556869409238
all trusts:  [0.06896551724137931]
max trusts:  0.06896551724137931
bid trusts:  75   0.0051813471502590676
total trusts:  0.07414686439163838
all trusts:  [0.1724137931034483]
max trusts:  0.1724137931034483
bid trusts:  45   0.003787878787878788
total trusts:  0.17620167189132707
all trusts:  [0.12643678160919541]
max trusts:  0.12643678160919541
bid trusts:  55   0.001941747572815534
total trusts:  0.12837852918201095
all trusts:  [0.40229885057471265, 0.20689655172413793, 0.20689655172413793]
max trusts:  0.40229885057471265
bid trusts:  13   0.0010050251256281408
total trusts:  0.4033038757003408
all trusts:  [0.11494252873563218]
max trusts:  0.11494252873563218
bid trusts:  28   0.002183406113537118
total trusts:  0.1171259348491693
all trusts:  [0.10344827586206896]
max trusts:  0.10344827586206896
bid trusts:  22   0.00546448087431694
total trusts:  0.10891275673638591
all trusts:  [0.06896551724137931, 0.

bid trusts:  40   0.005405405405405406
total trusts:  0.14333643988816402
all trusts:  [0.05747126436781609, 0.05747126436781609]
max trusts:  0.05747126436781609
bid trusts:  29   0.0005162622612287042
total trusts:  0.05798752662904479
all trusts:  [0.22988505747126436]
max trusts:  0.22988505747126436
bid trusts:  11   0.0035335689045936395
total trusts:  0.233418626375858
all trusts:  [0.1724137931034483]
max trusts:  0.1724137931034483
bid trusts:  45   0.003787878787878788
total trusts:  0.17620167189132707
all trusts:  [0.14942528735632185, 0.14942528735632185]
max trusts:  0.14942528735632185
bid trusts:  94   0.0018018018018018018
total trusts:  0.15122708915812366
all trusts:  [1.0, 1.0]
max trusts:  1.0
bid trusts:  2   0.002036659877800407
total trusts:  1.0020366598778003
all trusts:  [1.0, 1.0]
max trusts:  1.0
bid trusts:  27   0.0036363636363636364
total trusts:  1.0036363636363637
all trusts:  [1.0, 1.0]
max trusts:  1.0
bid trusts:  23   0.002512562814070352
total tru

bid trusts:  1   0.003676470588235294
total trusts:  0.2680442866801893
all trusts:  [0.11494252873563218]
max trusts:  0.11494252873563218
bid trusts:  31   0.00546448087431694
total trusts:  0.12040700960994913
all trusts:  [0.16091954022988506]
max trusts:  0.16091954022988506
bid trusts:  46   0.0078125
total trusts:  0.16873204022988506
all trusts:  [0.11494252873563218]
max trusts:  0.11494252873563218
bid trusts:  100   0.02564102564102564
total trusts:  0.14058355437665782
all trusts:  [0.13793103448275862]
max trusts:  0.13793103448275862
bid trusts:  40   0.005405405405405406
total trusts:  0.14333643988816402
all trusts:  [0.1839080459770115, 0.1839080459770115]
max trusts:  0.1839080459770115
bid trusts:  16   0.0036496350364963502
total trusts:  0.18755768101350784
all trusts:  [0.26436781609195403]
max trusts:  0.26436781609195403
bid trusts:  1   0.003676470588235294
total trusts:  0.2680442866801893
all trusts:  [0.26436781609195403]
max trusts:  0.26436781609195403
bid

In [19]:
print(new_books_trust)

{26: 1.002857142857143, 33: 0.523063973063973, 70: 0.300760582010582, 18: 1.002659574468085, 27: 1.0036363636363637, 21: 1.003257328990228, 2: 1.0020366598778003, 23: 1.0025125628140703, 24: 1.0030120481927711, 35: 0.12862018772273254, 84: 0.05165882967607106, 58: 0.046416185846470705, 5: 0.15016275048316552, 8: 0.2096743295019157, 65: 0.10759765345542996, 45: 0.17620167189132707, 14: 0.4034149220032841, 55: 0.12837852918201095, 36: 0.06267959770114942, 13: 0.4033038757003408, 47: 0.08444383385996244, 50: 0.18314176245210728, 87: 0.05515132342085838, 86: 0.02931761967117707, 32: 0.1291177467566485, 28: 0.1171259348491693, 42: 0.08104559319637193, 11: 0.233418626375858, 9: 1.0032154340836013, 81: 0.0683408295852074, 60: 0.13190126248351236, 10: 0.24166874594540647, 76: 0.2418871823610795, 94: 0.15122708915812366, 4: 0.1974556869409238, 100: 0.14058355437665782, 92: 0.031053021876158694, 71: 0.03486472959089592, 68: 0.042295258620689655, 48: 0.08243215670271373, 95: 0.06939973348106668, 

In [37]:
def rmse_mymethod(actual,predicted):
    sumsq=0.0
    count=0
    for i,row in enumerate(actual):
        for j,cell in enumerate(row):
            if(actual[i][j]!=0):
                count+=1
                sumsq+=((predicted[i][j]-actual[i][j])**2)
                #print(i,j,actual[i][j])
    print("count: ",count)
    return np.sqrt(sumsq/count)
def mae_mymethod(actual,predicted):
    sum_abs=0.0
    count=0
    for i,row in enumerate(actual):
        for j,cell in enumerate(row):
            if(actual[i][j]!=0):
                count+=1
                sum_abs+=abs(predicted[i][j]-actual[i][j])
                #print(i,j,actual[i][j])
    return(sum_abs/count)
def rmse(actual,predicted):
    return np.sqrt(((predicted-actual)**2).mean())


In [38]:
#normalizing trust
import numpy as np
arr_trust=np.array(list(new_books_trust.values()))
mean_trust=np.mean(arr_trust)
sd_trust=np.std(arr_trust)
#print(np.mean(new_books_trust.values()))

In [39]:
normalize_books_trust={}
for key,value in new_books_trust.items():
    normalize_books_trust[key]=(value-mean_trust)/sd_trust

In [40]:
print("before normal: ",new_books_trust)
print("after normal: ",normalize_books_trust)

before normal:  {26: 1.002857142857143, 33: 0.523063973063973, 70: 0.300760582010582, 18: 1.002659574468085, 27: 1.0036363636363637, 21: 1.003257328990228, 2: 1.0020366598778003, 23: 1.0025125628140703, 24: 1.0030120481927711, 35: 0.12862018772273254, 84: 0.05165882967607106, 58: 0.046416185846470705, 5: 0.15016275048316552, 8: 0.2096743295019157, 65: 0.10759765345542996, 45: 0.17620167189132707, 14: 0.4034149220032841, 55: 0.12837852918201095, 36: 0.06267959770114942, 13: 0.4033038757003408, 47: 0.08444383385996244, 50: 0.18314176245210728, 87: 0.05515132342085838, 86: 0.02931761967117707, 32: 0.1291177467566485, 28: 0.1171259348491693, 42: 0.08104559319637193, 11: 0.233418626375858, 9: 1.0032154340836013, 81: 0.0683408295852074, 60: 0.13190126248351236, 10: 0.24166874594540647, 76: 0.2418871823610795, 94: 0.15122708915812366, 4: 0.1974556869409238, 100: 0.14058355437665782, 92: 0.031053021876158694, 71: 0.03486472959089592, 68: 0.042295258620689655, 48: 0.08243215670271373, 95: 0.069

In [41]:
sim_trust_mtx=initialize_sim_mtx('jacra')
construct_sim_mtx(sim_trust_mtx,'jacra')
print(sim_trust_mtx)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
done
[[0.         1.70042145 0.55662736 ... 0.36621996 0.20736612 0.10418729]
 [0.52768647 0.         0.36433791 ... 0.33444745 0.06110475 0.26255197]
 [0.64495013 1.36033716 0.         ... 0.33444745 0.04875379 0.1500297 ]
 ...
 [0.51361483 1.51148574 0.4048199  ... 0.         0.06110475 0.16669966]
 [0.74814215 0.7103983  0.15180746 ... 0.1571903  0.         0.12502475]
 [0.29315915 2.38059004 0.36433791 ... 0.33444745 0.09750758 0.        ]]


In [43]:
predicted_ratings_matrix=get_pred(actualRatings_mtx,'jacra')
print("predicted: ",predicted_ratings_matrix)

predicting for jacra
actual for jacra  [[0. 0. 0. ... 0. 3. 4.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 3. 0. ... 0. 0. 0.]]
predicted for jacra  [[0. 0. 0. ... 0. 3. 4.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 3. 0. ... 0. 0. 0.]]
predicted:  [[0.         0.         0.         ... 3.77682977 3.75721716 3.76025826]
 [3.76025826 3.76025826 3.76025826 ... 4.         4.         4.        ]
 [4.         4.         4.         ... 1.93182079 1.93182079 1.93182079]
 ...
 [3.74349445 3.74349445 3.74349445 ... 4.01554175 4.01554175 4.01554175]
 [4.01554175 4.01554175 4.01554175 ... 3.43219761 3.43219761 3.43219761]
 [3.43219761 3.         3.         ... 3.25604653 3.25604653 3.25604653]]


c:\users\nived\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars


In [45]:
predicted_ratings_matrix=np.nan_to_num(predicted_ratings_matrix)
predicted_ratings_matrix

array([[0.        , 0.        , 0.        , ..., 3.77682977, 3.75721716,
        3.76025826],
       [3.76025826, 3.76025826, 3.76025826, ..., 4.        , 4.        ,
        4.        ],
       [4.        , 4.        , 4.        , ..., 1.93182079, 1.93182079,
        1.93182079],
       ...,
       [3.74349445, 3.74349445, 3.74349445, ..., 4.01554175, 4.01554175,
        4.01554175],
       [4.01554175, 4.01554175, 4.01554175, ..., 3.43219761, 3.43219761,
        3.43219761],
       [3.43219761, 3.        , 3.        , ..., 3.25604653, 3.25604653,
        3.25604653]])

In [46]:
print("auth+jacra rmse my method: ", rmse_mymethod(actualRatings_mtx,predicted_ratings_matrix))
print("auth+jacra rmse: ", rmse(actualRatings_mtx,predicted_ratings_matrix))

count:  674
auth+jaccard rmse my method:  0.9601778952853522
auth+jaccard rmse:  3.4174661589393116


In [47]:
print("auth+jacra mae my method: ", mae_mymethod(actualRatings_mtx,predicted_ratings_matrix))

auth+jaccard mae my method:  0.6959377615336629


In [ ]:
print(predicted_ratings_matrix)